In [12]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from os import listdir
from skimage.filters import sobel, prewitt, laplace
from skimage.feature import canny
from skimage.color import rgb2hed
from skimage.exposure import rescale_intensity
from skimage.color import rgb2gray

In [15]:
if __name__=='__main__':
    main()

In [ ]:
folder = "3/"
names = [f for f in listdir(folder)]
for name in names:
    process(folder + name)

[[492, 342, 410, 327]]
[492, 342, 410, 327]
[[626, 505, 582, 512], [429, 298, 547, 470], [205, 93, 506, 447], [577, 450, 409, 341], [785, 665, 374, 314], [447, 319, 346, 273], [206, 94, 319, 261]]
[626, 505, 582, 512]
[429, 298, 547, 470]
[205, 93, 506, 447]
[577, 450, 409, 341]
[785, 665, 374, 314]
[447, 319, 346, 273]
[206, 94, 319, 261]
[[334, 242, 600, 547], [182, 94, 546, 496], [585, 492, 521, 470], [372, 271, 476, 423], [142, 52, 424, 377], [458, 362, 365, 312]]
[334, 242, 600, 547]
[182, 94, 546, 496]
[585, 492, 521, 470]
[372, 271, 476, 423]
[142, 52, 424, 377]
[458, 362, 365, 312]
[[462, 300, 575, 495], [259, 65, 507, 457], [727, 581, 496, 417], [535, 352, 444, 404], [321, 172, 386, 307], [557, 419, 331, 260]]
[462, 300, 575, 495]
[259, 65, 507, 457]
[727, 581, 496, 417]
[535, 352, 444, 404]
[321, 172, 386, 307]
[557, 419, 331, 260]


In [13]:
def process(name):   
    filename = name #z trudnego 13,16, 24, 25, 26, 32, 37, 40 - nie dziala (mały błąd)
    img = cv2.imread(filename)
    image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (960, 720))
    img = cv2.resize(img, (960, 720))

    image2 = image


    x = round(np.std(image) + np.mean(image))
    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i][j] > x:
                image[i][j] = 255
            else:
                image[i][j] = 0


    kernel = np.ones((5,5), np.uint8) 
    image = cv2.erode(image, kernel, iterations=1) 
    #image = np.uint8(image * 255)

    if False:
        cv2.imshow('Kostki', image)
        cv2.waitKey()
        cv2.destroyAllWindows()


    # szukam kostek
    p = []
    x = []
    y = []
    cubes = []
    d = 0
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #print(hierarchy)
    for i in range(len(contours)):
        if len(contours[i]) > 30 and len(contours[i]) < 242 and hierarchy[0][i][2] != -1:
            # print(hierarchy[0][i][2])
            # print(len(contours[i]))
            for j in range(len(contours[i])):
                x.append(contours[i][j][0][0])
                y.append(contours[i][j][0][1])
            cubes.append([max(x), min(x), max(y), min(y)])
            x = []
            y = []

    # wycinam kostki i znajduje koła i obliczam ile oczek    

    suma = 0
    szesc = []
    tab_suma = []
    do_kostek = []
    print(cubes)
    kernel = np.ones((3,3), np.uint8) 
    for j in range(len(cubes)):
        print(cubes[j])
        ie = image[cubes[j][3]:cubes[j][2], cubes[j][1]:cubes[j][0]]
        ie_col = img[cubes[j][3]:cubes[j][2], cubes[j][1]:cubes[j][0]]
        ie = cv2.resize(ie, (400, 400))
        ie_col = cv2.resize(ie_col, (400, 400))
        ie = cv2.erode(ie, kernel, iterations=2) 
        #ie = cv2.dilate(ie, kernel, iterations=1)
        contours_cube, hierarchy_cube = cv2.findContours(ie, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


        sum = 0   
        do_maxa = []

        for i in range(len(contours_cube)):
            do_maxa.append(len(contours_cube[i]))
        max_len = max(do_maxa)
        do_kostek.append(max_len)
        pom = []
        for i in range(len(contours_cube)):
            #print(len(contours_cube[i]))
            if  len(contours_cube[i]) < max_len and len(contours_cube[i]) > 30:
                sum = sum + 1
                suma = suma + 1
                cv2.drawContours(ie_col, contours_cube, i, (0,255,0), 3)
                pom.append(len(contours_cube[i]))
                p.append(j) 
        szesc.append(pom)
        tab_suma.append([j, sum])
        if j in p:
            cv2.rectangle(img,(cubes[j][1], cubes[j][3]),(cubes[j][0], cubes[j][2]),(0,255,0),3)

        if False:
            cv2.imshow('Kostki', ie_col)
            cv2.waitKey()
            cv2.destroyAllWindows()

    szesc = [x for x in szesc if x != []]        
    #print(szesc)   
    a = min(szesc)
    b = max(szesc)

    c = do_kostek[szesc.index(b)]
    b_m = max(b)
    #print(a,b,c)
    for i in range(len(tab_suma)):
        if (b_m > (2.5*a[0]) and len(szesc) != 1) and i == szesc.index(b) and tab_suma[i][1] == 2:
            cv2.putText(img, str(tab_suma[i][1] * 3), (cubes[tab_suma[i][0]][0], 
                                            cubes[tab_suma[i][0]][2]), cv2.FONT_HERSHEY_PLAIN, 3,  (80, 0, 200, 255), thickness = 3)   
            suma = suma + 4
        else:
            if tab_suma[i][1] != 0:
                cv2.putText(img, str(tab_suma[i][1]), (cubes[tab_suma[i][0]][0], 
                                            cubes[tab_suma[i][0]][2]), cv2.FONT_HERSHEY_PLAIN, 3,  (80, 0, 200, 255), thickness = 3)    
    cv2.putText(img, "Liczba oczek: {}".format(suma), (10, 35), 
                       cv2.FONT_HERSHEY_PLAIN, 3, (50, 0, 255, 255), thickness = 4)

    for i in range(len(contours)):
        for j in p:
            if hierarchy[0][i][3] == j:
                sum = sum + 1
                p.append(i)

    # rysuje kontury 
    #print(sum)  
    #for j in p:
    #    cv2.drawContours(img, contours, j, (0,255,0), 3)



    cv2.imshow('Kostki',img)
    cv2.waitKey()
    cv2.destroyAllWindows()
